# Seq2Seq Learning

## 数据准备

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Anaconda2\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64  # 批大小
epochs = 100     # 迭代次数
latent_dim = 256 # 隐含层节点个数
n_samples = 10000 # 样本数量

# 数据路径
data_path = './cmn-eng/cmn.txt'

In [3]:
# 数据处理
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[: min(n_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    
    # 目标句子以'\t'作为开始字符，'\n'结束
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # 将input的字符放入input字符集
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    
    # 将target的字符放入target字符集
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

# 将集合中的字符进行排序
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

n_encoder_tokens = len(input_chars)  # input字符集大小
n_decoder_tokens = len(target_chars) # target字符集大小 
max_encoder_seq_length = max([len(txt) for txt in input_texts])  # input字符串的最大长度
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # target字符串的最大长度

In [4]:
print('样本数量：', len(input_texts))
print('不同的输入字符个数：', n_encoder_tokens)
print('不同的输出字符个数：', n_decoder_tokens)
print('输入序列最大长度：', max_encoder_seq_length)
print('输出序列最大长度：', max_decoder_seq_length)

样本数量： 10000
不同的输入字符个数： 73
不同的输出字符个数： 2623
输入序列最大长度： 30
输出序列最大长度： 22


In [5]:
# 构造字符集字典，每个字符对应一个编号，如 'H'->0, 'e'->1, ...

# 构造input字符集字典
input_token_dict = dict(
    [(char, i) for i, char in enumerate(input_chars)])
# 构造target字符集字典
target_token_dict = dict(
    [(char, i) for i, char in enumerate(target_chars)])

In [6]:
# 对序列每个时刻字符的对应位置设为1

# 数据初始化，形状：(样本数量 x 序列最大长度 x 字符个数)
# 初始化编码的输入数据
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, n_encoder_tokens), 
    dtype='float32')

# 初始化解码的输入数据
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, n_decoder_tokens), 
    dtype='float32')

# 初始化解码的输出数据
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, n_decoder_tokens), 
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # 将编码输入数据t时刻的对应字符的位置设为1
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_dict[char]] = 1
        
    for t, char in enumerate(target_text):
        # 将解码输入数据t时刻的对应字符的位置设为1
        decoder_input_data[i, t, target_token_dict[char]] = 1
        if t > 0:
            # 将解码输出数据t-1时刻的对应字符的位置设为1
            decoder_target_data[i, t - 1, target_token_dict[char]] = 1

## 模型搭建

### 训练阶段

In [7]:
# 定义编码模型
encoder_inputs = Input(shape=(None, n_encoder_tokens), name='enc_input_layer')
encoder_lstm = LSTM(latent_dim, return_state=True, name='enc_lstm_layer')
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [8]:
# 定义解码模型
decoder_inputs = Input(shape=(None, n_decoder_tokens), name='dec_input_layer')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='dec_lstm_layer')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(n_decoder_tokens, activation='softmax', name='dec_dense_layer')
decoder_outputs = decoder_dense(decoder_outputs)

In [9]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
enc_input_layer (InputLayer)    (None, None, 73)     0                                            
__________________________________________________________________________________________________
dec_input_layer (InputLayer)    (None, None, 2623)   0                                            
__________________________________________________________________________________________________
enc_lstm_layer (LSTM)           [(None, 256), (None, 337920      enc_input_layer[0][0]            
__________________________________________________________________________________________________
dec_lstm_layer (LSTM)           [(None, None, 256),  2949120     dec_input_layer[0][0]            
                                                                 enc_lstm_layer[0][1]             
          

In [10]:
# 训练模型
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size, epochs=epochs, validation_split=0.2)

# 保存模型
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 109s 14ms/step - loss: 2.0339 - val_loss: 2.4937
Epoch 2/100
8000/8000 [==============================] - 103s 13ms/step - loss: 1.8995 - val_loss: 2.4177
Epoch 3/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.7902 - val_loss: 2.3173
Epoch 4/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.7137 - val_loss: 2.2279
Epoch 5/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.6423 - val_loss: 2.1755
Epoch 6/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.5857 - val_loss: 2.1170
Epoch 7/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.5287 - val_loss: 2.0744
Epoch 8/100
8000/8000 [==============================] - 102s 13ms/step - loss: 1.4812 - val_loss: 2.0511
Epoch 9/100
8000/8000 [==============================] - 105s 13ms/step - loss: 1.4276 - val_loss: 1.986

C:\Anaconda2\envs\py35\lib\site-packages\keras\engine\topology.py:2368: UserWarning: Layer dec_lstm_layer was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'enc_lstm_layer/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'enc_lstm_layer/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


### 预测阶段

In [11]:
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 256 # 隐含层节点个数

model = load_model('s2s.h5')

# 加载编码模型
encoder_inputs = model.inputs[0]
encoder_lstm = model.get_layer('enc_lstm_layer')
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)

# 加载解码模型
decoder_inputs = Input(shape=(None, n_decoder_tokens))
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.get_layer('dec_lstm_layer')
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_dense = model.get_layer('dec_dense_layer')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [12]:
# 构建反向字符集字典，如：0->'H', 1->'e', ...
reverse_target_char_dict = dict((i, char) for char, i in target_token_dict.items())

In [13]:
def decode_sequence(input_seq):
    """
        根据输入字符串，输出翻译的字符串
    """
    # 将input编码为 state_h 和 state_c
    states_value = encoder_model.predict(input_seq)

    # 初始化target序列
    target_seq = np.zeros((1, 1, n_decoder_tokens))
    # 将第一个字符设为'\t'
    target_seq[0, 0, target_token_dict['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 根据target_seq的输入和states_value进行解码
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 找到概率最大的target字符的编号
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # 将编号转换为字符
        sampled_char = reverse_target_char_dict[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 更新target字符串
        target_seq = np.zeros((1, 1, n_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新状态
        states_value = [h, c]

    return decoded_sentence

In [14]:
for seq_index in range(100):
    # 从训练集中取出前100个样本
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('英文输入:', input_texts[seq_index])
    print('中文翻译:', decoded_sentence)

-
英文输入: Hi.
中文翻译: 你好。

-
英文输入: Hi.
中文翻译: 你好。

-
英文输入: Run.
中文翻译: 你用跑的。

-
英文输入: Wait!
中文翻译: 等等！

-
英文输入: Hello!
中文翻译: 你好。

-
英文输入: I try.
中文翻译: 让我来。

-
英文输入: I won!
中文翻译: 我赢了。

-
英文输入: Oh no!
中文翻译: 不会吧。

-
英文输入: Cheers!
中文翻译: 乾杯!

-
英文输入: He ran.
中文翻译: 他跑了。

-
英文输入: Hop in.
中文翻译: 跳进来。

-
英文输入: I lost.
中文翻译: 我迷失了。

-
英文输入: I quit.
中文翻译: 我退出。

-
英文输入: I'm OK.
中文翻译: 我沒事。

-
英文输入: Listen.
中文翻译: 听着。

-
英文输入: No way!
中文翻译: 不可能！

-
英文输入: No way!
中文翻译: 不可能！

-
英文输入: Really?
中文翻译: 你确定？

-
英文输入: Try it.
中文翻译: 试试吧。

-
英文输入: We try.
中文翻译: 我们来试试。

-
英文输入: Why me?
中文翻译: 为什么是我？

-
英文输入: Ask Tom.
中文翻译: 去问汤姆。

-
英文输入: Be calm.
中文翻译: 冷静点。

-
英文输入: Be fair.
中文翻译: 公平点。

-
英文输入: Be kind.
中文翻译: 友善点。

-
英文输入: Be nice.
中文翻译: 和气点。

-
英文输入: Call me.
中文翻译: 联系我。

-
英文输入: Call us.
中文翻译: 联系我们。

-
英文输入: Come in.
中文翻译: 进来。

-
英文输入: Get Tom.
中文翻译: 找到汤姆。

-
英文输入: Get out!
中文翻译: 滾出去！

-
英文输入: Go away!
中文翻译: 走開！

-
英文输入: Go away!
中文翻译: 走開！

-
英文输入: Go away.
中文翻译: 走開！

-
英文输入: Goodbye!
中文翻译: 再见！

-
英文输入: Goodbye!
中文翻译: 再见